# Exercise - Encoding

In [ ]:
# Install matplotlib for plotting results. Numpy is included in the package
# Pillow is the basic Image library for Python, and will be used for showing
# the image-like 2-D results
# array2gif will be used for creating RGB animated gifs
!pip install matplotlib
!pip install pillow
!pip install array2gif

# Download a cat test image from Wikipedia
!wget --output-document=cat.jpg https://i.pinimg.com/280x280_RS/3a/d2/cc/3ad2cc3dea6225983487904da4be52f4.jpg


# Frequency encoding: Single value conversion

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In this section a method for implementing *frequency encoding* will be introduced. The solution is entirely done in Python, and it only requires the *NumPy* library. Additionally, *matplotlib* is used for showing the results. Please notice that this kind of encoding is often called *rate encoding* in literature., and both terms are equivalent. We will use *frequency encoding* for the rest of this exercise.

The following function is used for generating the spike train parameters. As explained in the course slides, it basically consists in a scale conversion
from the value domain to the frequency domain.

In [ ]:
def get_spike_params(value, min_frequency=0.1, max_frequency=1, min_value=0,
                     max_value=255, max_time=20, t_0=0):
    """
    Obtain the spike train parameters for a frequency encoder.

    The generated parameters are the period, initial spike time, and the
    total number of spikes that have to be generated for the specified
    time range.
    """
    input_range = max_value - min_value
    frequency_range = max_frequency - min_frequency
    scale_factor = frequency_range / input_range
    frequency = min_frequency + (value-min_value)*scale_factor
    period = 1 / frequency
    # Generate the first spike at a random position within the range of the
    # obtained period
    init_spike = np.random.uniform(t_0, t_0+period)

    # Calculate the spike times and generate the spike train
    n_spikes = np.trunc((max_time-init_spike)*frequency + 1)    
    return (period, init_spike, n_spikes)

On top of the previous function, we use another function that takes into account the specific constraints of single floating values. As you will see later in this exercise, in case of 2-D images it will be necessary to use slightly different instructions.

To sum up, the following function takes the spike train parameters and build a spike train itself. From a data structure point of view, this spike train consists of a list of values that represent the time at which each specific spike occurs.

In [ ]:
def value2spikes(value, min_value=0, max_value=100, **kwargs):
    """
    Convert one single float value into a spike train.
    """
    # Make sure that the provided value is coherent with 
    # the provided limits
    if value>max_value or value<min_value:
        raise ValueError("Value off bounds")
    kwargs["min_value"] = min_value
    kwargs["max_value"] = max_value

    # Calculate the spiking parameters and the spike train
    period, init_spike, n_spikes = get_spike_params(value)
    spike_train = np.arange(n_spikes)*period + init_spike
    spike_train = np.around(spike_train, 1)
    return spike_train

Let's try the two functions that have just been introduced by feeding some values and plotting the resulting spike train.

In [ ]:
values = [10, 30, 95]
spike_trains = []
for value in values:
    spike_train = value2spikes(value)
    spike_trains.append(spike_train)
plt.eventplot(spike_trains, colors = ['r', 'g', 'b'])
plt.legend(['10', '30', '95'], bbox_to_anchor=(0.2, 0.8, 1, .1), loc=5, ncol=1)
ax = plt.gca()
ax.axes.yaxis.set_visible(False)

# Frequency encoding: 2-D image conversion

Now that we have a basic understanding on how to apply frequency encoding to a single float value between previously defined limits, let's apply this same concept to a 2D image. For the sake of simplicity, the input image will be converted to a grey scale, where each pixel has an intensity value ranged between 0 and 255 (For those not used to deal with images, this is the most conventional way of representing images in computer science experiments).

As a first step, we will use the following routine for reading our test image and converting it to the aforementioned format:

In [ ]:
def rgb2gray(rgb):
    """
    Convert input RGB image into a grey scale image.

    Obtained from https://stackoverflow.com/q/12201577/3982405
    """
    gray_img = np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])
    return gray_img

# Read the test image and convert it to a grey scale image
image = plt.imread("cat.jpg")
grey_image = rgb2gray(image)

# Show the original and grey scale images
_, imgaxis = plt.subplots(1,2)
imgaxis[0].axis('off')
imgaxis[1].axis('off')
imgaxis[0].imshow(image)
imgaxis[1].imshow(grey_image, cmap="gray", vmin=0, vmax=255)

Let's now adapt the function `value2frequency()` that we created before, so it can handle 2-D images. In order to do so, let's create another function called 
`generate_image_spiketrain` for generating a data structure containing the spike train corresponding to each pixel. This is not the most optimal way of achieving our goal from a computational perspective, but it is very easy to visualize in the code what we are exactly doing i.e. we are creating a vector per pixel where we store the spike times for that specific pixel.


In [ ]:
def generate_image_spiketrain(img, init_spikes, periods, n_spikes):
    result = np.zeros_like(img, dtype=object)
    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            spike_train = np.arange(n_spikes[row,col])*periods[row,col]
            spike_train += init_spikes[row,col]
            result[row, col] = spike_train.astype(np.int)

    return result




Next, let's implement the function `image2frequency()` that was mentioned before, which first calculates the spiking parameters pixel-wise and then create a spike train per pixel.

In [ ]:
def image2spikes(img, min_value=0, max_value=255, **kwargs):
    """
    Convert a 2-D grey scale image into pixel-wise spike trains.
    """
    # Make sure that the provided value is coherent with the limits
    if not (img>=min_value).all() and (img<=max_value).all():
        raise ValueError("Value off bounds")
    kwargs["min_value"] = min_value
    kwargs["max_value"] = max_value
    # Invert image values, so high values translate into high frequency
    img = 255-img

    # Calculate the spike train for every pixel in the image
    periods, init_spikes, n_spikes = get_spike_params(img, **kwargs)
    spike_train = generate_image_spiketrain(img, init_spikes, periods, n_spikes)
    return spike_train

Now that we are able to generate a spike train per pixel, we will feed our test image to these functions and arbitrary amount of timestamps. At each timestamp, we call the function `train2spikes()` in order to know which pixels shall spike.



In [ ]:
def train2spikes(spike_trains, timestep):
    """
    Decide for each pixel if there is a spike at the selected time step.
    """
    spikes = np.zeros_like(spike_trains).astype(np.int)
    # Iterate through all pixels and decide which ones spike
    for row in range(spikes.shape[0]):
        for col in range(spikes.shape[1]):
            # There is a spike if the input time step is contained in
            # the pixel spike times
            if timestep in spike_trains[row,col]:
                spikes[row,col] = 1
    return spikes

def create_image_sequence():
    spike_trains = image2spikes(grey_image)
    # Very high values in the amount of timesteps will considerably increase
    # the computational cost in the following steps.
    timesteps = np.arange(30)

    # Generate an image sequence, and decide which pixels spike
    # at each time step
    images = []
    for step in timesteps:
        images.append(train2spikes(spike_trains, step))
    return images

Finally, let's plot the obtained results and observe the effect of frequency encoding on 2-D images. Ideally, we would use the function `plt.imshow()` in an iterative fashion, but the output is not the desired in the *Jupyter Notebook*. Instead, we will download a small library (*array2gif*) for generating an animated gif and we will open it afterwards.

In [ ]:
from array2gif import write_gif
from IPython.display import Image


images = create_image_sequence()
# Create a 3-channels image from the grey original. It's still a grey image, 
# but the RGB format is required for using the array2gif library
rgb_formatted = []
for img in images:
    rgb_image = np.stack((img.transpose()*255,)*3, axis=-1)
    rgb_formatted.append(rgb_image)
write_gif(rgb_formatted, 'result.gif', fps=3)

Image(open('result.gif','rb').read())



# Open questions


*   How does the spikes change when we change the frequency maximum and minimumy values? Try modifying those values when calling the function image2frequency in the input line [8]
*   There are other encoding techniques that have been introduced in the lecture slides. Try modifying the `get_spike_params()` in the input line [4] in order to implement temporal encoding. An easy approach would be to use *time to first spike* encoding technique, so the highest the value the faster it spikes. How does it look now? (In the exercise 3 you can get an idea of how it should look)



